## Estimating individual systematic contributions

The effect of any individual systematic uncertainty is somewhat complicated by it's mutual covariance with the POI (i.e., the W branching fractions) and any other systematic uncertainty.  To get a rough idea of the percent-wise contribution to the total uncertainty from each individual systematic, I use the following scheme:

   * the fit is carried out as in the nominal case and $\sigma_{0}$ is estimated
   * the fit is carried out for for each of the $n$ nuisance parameters $\sigma_{theta}$
   * the difference between the nominal case and the $n-1$ case is calculated,
   * this quantity is normalized to $\sum_{\theta} \sigma_{\theta}^{2}$

In [1]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from multiprocessing import Pool
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import minimize
from tqdm import tqdm_notebook, trange

import scripts.plot_tools as pt
import scripts.fit_helpers as fh

np.set_printoptions(precision=3)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':18,
             'ytick.labelsize':18,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 40111,
  "iopub_port": 49323,
  "stdin_port": 34903,
  "control_port": 33023,
  "hb_port": 52823,
  "ip": "127.0.0.1",
  "key": "f5b30996-3372abef82c6089eff515cf1",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d0227585-deb5-4cf8-b0dc-72b49c561a46.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# configure, get the input data, and do any additional processing that is needed
processes = ['ttbar', 't', 'ww', 'wjets', 'zjets_alt', 'diboson', 'fakes'] 
selections = [
              'ee',  'mumu',  
              'emu', 
              'mutau', 'etau', 
              'mu4j', 'e4j'
             ]
plot_labels = fh.fancy_labels

# initialize fit data
infile = open('local_data/fit_data_postfit.pkl', 'rb')
fit_data = pickle.load(infile)
infile.close()

# get fit parameters
parameters = pd.read_csv('data/model_parameters_asimov.csv', index_col='name')

In [3]:
# prepare Asimov dataset
asimov_data = {cat:fit_data.mixture_model(parameters.val_init.values, cat) for cat in fit_data._model_data.keys()}

In [4]:
# initialize veto list
fit_data.veto_list = [
    # baseline
    #'ee_cat_gt2_eq1_b', 'ee_cat_gt2_gt2_b', 
    #'mumu_cat_gt2_eq1_b', 'mumu_cat_gt2_gt2_b', 
    #'emu_cat_gt2_eq1_a', 'emu_cat_gt2_gt2_a', 
    #'etau_cat_eq2_eq1', 'etau_cat_gt3_eq1', 'etau_cat_eq2_gt2', 'etau_cat_gt3_gt2', 
    #'mutau_cat_eq2_eq1', 'mutau_cat_gt3_eq1', 'mutau_cat_eq2_gt2', 'mutau_cat_gt3_gt2', 
    #'e4j_cat_gt4_eq1', 'e4j_cat_gt4_gt2',
    #'mu4j_cat_gt4_eq1', 'mu4j_cat_gt4_gt2', 
    'e4j_cat_eq3_gt2', 'mu4j_cat_eq3_gt2',
    
    # e/mu DY CR
    'ee_cat_gt2_eq0',  'mumu_cat_gt2_eq0', 
    
    # e+mu additional ttbar
    #'emu_cat_gt2_eq0', 'emu_cat_eq1_eq0_a', 'emu_cat_eq1_eq1_a', 
    
    # e+mu WW
    #'emu_cat_eq0_eq0_a', 
    
    # e/mu+tau additional CR
    #'mutau_cat_eq0_eq0', 'mutau_cat_eq1_eq0', 
    #'mutau_cat_gt2_eq0', 'mutau_cat_eq1_eq1', 
    #'etau_cat_eq0_eq0', 'etau_cat_eq1_eq0', 
    #'etau_cat_gt2_eq0', 'etau_cat_eq1_eq1', 
]

In [5]:
# fit configuration #

# minimizer options
steps = 4*[1e-4, ] 
steps += list(0.05*fit_data._perr_init[4: fit_data._nnorm + 7])
steps += list(0.05*fit_data._perr_init[fit_data._npoi + fit_data._nnorm:])
steps = np.array(steps)
min_options = dict(
                   #finite_diff_rel_step=steps,
                   #verbose=3,
                   #eps=1e-9, 
                   gtol = 1e-3,
                   disp = True
                  )

# configure the objective
sample = asimov_data
fobj = partial(fit_data.objective,
               data = sample,
               do_bb_lite = True,
               lu_test = None
              )

fobj_jac = partial(fit_data.objective_jacobian,
                   data = sample,
                   do_bb_lite = True,
                   lu_test = None
                  )


In [6]:
# n-1 systematics

# initialize parameter data
fit_data._pmask = parameters['active'].values.astype(bool)
fit_data._pval_init = parameters['val_init'].values.copy()

errs = []
impacts = []
iparam = 4
mask = fit_data._pmask
#mask[17:] = False
p_init = fit_data._pval_fit
p_fit = parameters.val_fit.values
for pname, pdata in tqdm_notebook(parameters.iloc[4:].iterrows(), total=parameters.active.sum() - 4):
    if not pdata.active:
        iparam += 1
        continue
        
    print(iparam, pname, pdata.val_init, pdata.err_init, pdata.val_fit, pdata.err_fit, fobj(p_init[mask]))

    # fix parameter in fit and covariance calculation
    mask[iparam] = False
    #min_options['finite_diff_rel_step'] = steps[mask]
    err, _ = fh.calculate_covariance(fobj, p_init[mask])
    errs.append(err)
    
    # calculate impacts from up/down variations of n.p. on p.o.i.
    p_init[iparam] = pdata.val_fit + pdata.err_fit
    print(p_init[iparam], fobj(p_init[mask]))
    res = minimize(fobj, p_fit[mask],
                   jac     = fobj_jac,
                   #hess    = 'cs',
                   #method  = 'trust-constr', 
                   method  = 'BFGS', 
                   options = min_options,
                  )
    impact_up = res.x
    print((res.x[:4]-p_fit[:4])*100/p_fit[:4], res.fun)
                    
    p_init[iparam] = pdata.val_fit - pdata.err_fit
    print(p_init[iparam], fobj(p_init[mask]))
    res = minimize(fobj, p_fit[mask],
                   jac     = fobj_jac,
                   #hess    = 'cs',
                   #method  = 'trust-constr', 
                   method  = 'BFGS', 
                   options = min_options,
                  )
    impact_down = res.x
    print((res.x[:4]-p_fit[:4])*100/p_fit[:4], res.fun)
    impacts.append(np.array([impact_up, impact_down]))
    
    p_init[iparam] = pdata.val_fit
    mask[iparam] = True
    
    iparam += 1
    #if iparam > 15:
    #    break
    
errs = np.array(errs)
impacts = np.array(impacts)

4 br_tau_e 0.1774 0.0004 0.1774 0.0003999512734901713 5.463594021648532e-10


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.1777999512734902 0.5081957400672589
Optimization terminated successfully.
         Current function value: 0.500001
         Iterations: 40
         Function evaluations: 80
         Gradient evaluations: 79
[-0.013  0.002  0.003  0.001] 0.5000006694110887
0.17700004872650982 0.5081968055919831
         Current function value: 0.500001
         Iterations: 40
         Function evaluations: 125
         Gradient evaluations: 113
[ 0.013 -0.002 -0.003 -0.001] 0.5000010476933358
5 br_tau_mu 0.173 0.0004 0.173 0.00039933807887907634 5.463594021648532e-10


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.17339933807887906 0.5175974850005058
         Current function value: 0.500001
         Iterations: 44
         Function evaluations: 101
         Gradient evaluations: 91
[-0.003 -0.012 -0.042  0.009] 0.5000006216220048
0.17260066192112092 0.5176024107490416
Optimization terminated successfully.
         Current function value: 0.500003
         Iterations: 43
         Function evaluations: 61
         Gradient evaluations: 61
[ 0.003  0.012  0.041 -0.009] 0.5000029346402786
6 br_tau_h 0.6496 0.001 0.6496 0.000999238121665807 5.463594021648532e-10


...

0.9803146448229632 22.512838993018512
         Current function value: 0.514552
         Iterations: 48
         Function evaluations: 127
         Gradient evaluations: 116
[-0.01  -0.02  -0.048  0.012] 0.5145519354293177
-0.9803146448229632 22.15725981851212
         Current function value: 0.503469
         Iterations: 51
         Function evaluations: 132
         Gradient evaluations: 120
[ 0.006  0.01   0.033 -0.008] 0.5034686473232144
114 jes_single_pion_hcal 0.0 1.0 0.0 0.98625619240705 5.463594021648532e-10


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.98625619240705 17.380604435519256
Optimization terminated successfully.
         Current function value: 0.515949
         Iterations: 50
         Function evaluations: 71
         Gradient evaluations: 71
[-0.01  -0.012 -0.035  0.009] 0.5159489955534079
-0.98625619240705 17.16807487591817
         Current function value: 0.506046
         Iterations: 51
         Function evaluations: 270
         Gradient evaluations: 256
[ 0.001  0.002  0.012 -0.002] 0.5060459412688509
115 jes_time_pt_eta 0.0 1.0 0.0 0.9942703400123796 5.463594021648532e-10


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.9942703400123796 1.0648741509525106
Optimization terminated successfully.
         Current function value: 0.508336
         Iterations: 46
         Function evaluations: 63
         Gradient evaluations: 63
[ 0.017  0.021  0.021 -0.009] 0.5083361387864881
-0.9942703400123796 1.0466973913910724
Optimization terminated successfully.
         Current function value: 0.500655
         Iterations: 43
         Function evaluations: 63
         Gradient evaluations: 63
[-0.008 -0.009  0.01   0.001] 0.500655088884262
116 jes_flavor_qcd 0.0 1.0 0.0 0.7202563265634524 5.463594021648532e-10
0.7202563265634524 253.99582789641678
         Current function value: 0.506225
         Iterations: 61
         Function evaluations: 134
         Gradient evaluations: 123
[ 0.133  0.14   0.133 -0.065] 0.5062246267567221
-0.7202563265634524 261.89139722364337
         Current function value: 0.524265
         Iterations: 57
         Function evaluations: 169
         Gradient evaluations: 159
[-0.139 -0.1

In [8]:
# convert impacts to dataframes 
impacts_np = np.hstack([100*(impacts[:,0,:4] - p_fit[:4])/p_fit[:4], 100*(impacts[:,1,:4] - p_fit[:4])/p_fit[:4]])

#impacts_up = pd.DataFrame(impacts_np, columns=['beta_e_up', 'beta_mu_up', 'beta_tau_up', 'beta_h_up'], index=list(p_labels_fancy[4:]))
p_labels = parameters.query('active == 1').index.values[4:]
impacts_np = pd.DataFrame(impacts_np, columns=['beta_e_up', 'beta_mu_up', 'beta_tau_up', 'beta_h_up', 'beta_e_down', 'beta_mu_down', 'beta_tau_down', 'beta_h_down'], index=list(p_labels))
pd.DataFrame(impacts_np)

,beta_e_up,beta_mu_up,beta_tau_up,beta_h_up,beta_e_down,beta_mu_down,beta_tau_down,beta_h_down
br_tau_e,-0.012725,0.002115,0.002977,0.001220,0.012720,-0.002123,-0.002982,-0.001217
br_tau_mu,-0.003195,-0.012014,-0.041577,0.009072,0.003206,0.012042,0.041439,-0.009057
br_tau_h,0.027804,0.023572,-0.110573,0.009457,-0.027870,-0.023613,0.110719,-0.009464
lumi,0.054880,-0.012637,0.059235,-0.016213,-0.058490,0.017161,-0.066687,0.017257
xs_diboson,-0.001507,-0.001005,-0.018107,0.003294,0.001553,0.001143,0.018564,-0.003397
xs_ww,-0.004323,-0.011831,-0.045521,0.009853,-0.003825,0.016334,0.065999,-0.012542
xs_t,0.140440,0.148294,0.245025,-0.085275,-0.150129,-0.160458,-0.253317,0.090091
xs_wjets,-0.058974,-0.053157,0.092028,0.003211,0.046699,0.039980,-0.112074,0.004058
e_fakes,-0.097671,0.072674,-0.163455,0.030108,0.099534,-0.071985,0.164166,-0.030629
mu_fakes,0.217152,0.140709,-0.128162,-0.036698,-0.219895,-0.140623,0.109069,0.040172


In [14]:
# convert to errors
errs_np = np.array([np.concatenate([e[:i+4], [0], e[i+4:]]) for i, e in enumerate(errs)])
#errs_np = err_no_bb**2 - errs_np**2
errs_np[errs_np < 0] = 0
errs_np = np.sqrt(errs_np)
#errs_np = np.vstack([errs_np, err_stat, err_mc_stat, err_syst, stderr])

errs_np = pd.DataFrame(errs_np[:,:4], columns=['beta_e', 'beta_mu', 'beta_tau', 'beta_h'], index=list(p_labels))
#errs_np = pd.DataFrame(errs_np[:,:4], columns=['beta_e', 'beta_mu', 'beta_tau', 'beta_h'], index=list(p_labels_fancy[4:]) + ['stat.', 'MC stat.', 'syst. total', 'total'])
#beta_stderr = stderr.iloc[:,:4].multiply(100)/params_init[:4]
100*errs_np/parameters

,beta_e,beta_mu,beta_tau,beta_h
br_tau_e,3.081983,2.650105,4.168588,4.781581
br_tau_mu,3.082137,2.649879,4.167895,4.780726
br_tau_h,3.081321,2.649196,4.163629,4.780630
lumi,3.078836,2.649772,4.167010,4.778588
xs_diboson,3.082125,2.650099,4.168428,4.781417
xs_ww,3.082126,2.649750,4.167200,4.780033
xs_t,3.059565,2.610399,4.143245,4.696620
xs_wjets,3.079104,2.646468,4.164403,4.781534
e_fakes,3.072253,2.641705,4.157663,4.771821
mu_fakes,3.031548,2.617427,4.162999,4.765015


In [ ]:
# print table

#beta_errs.divide(params_init[:4]/100, axis=1).to_latex('local_data/errors.tex')
#beta_errs.divide(params_init[:4]/100, axis=1).to_csv('local_data/errors.csv')
#beta_errs.divide(params_init[:4]/100, axis=1)
beta_errs.to_latex('local_data/summary_errors.tex', escape=False)
beta_errs

In [ ]:
jes_mask = np.array([True if ('jes' in pname and 'btag' not in pname) else False for pname in beta_errs.index])
btag_mask = np.array([True if 'btag' in pname else False for pname in beta_errs.index])
tau_misid_mask = np.array([True if ('misid_tau' in pname and pname not in ['misid_tau_e', 'misid_tau_h']) else False for pname in beta_errs.index])

btag_errs = beta_errs[btag_mask]
jes_errs = beta_errs[jes_mask]
tau_misid_errs = beta_errs[tau_misid_mask]

summary_errs = beta_errs[~btag_mask&~jes_mask&~tau_misid_mask].copy()
summary_errs.index = [fit_data._parameters.loc[p].label if p in fit_data._parameters.index else p for p in summary_errs.index]
summary_errs.loc['b-tag',:] = np.sqrt(np.sum(btag_errs**2))
summary_errs.loc['JES',:]  = np.sqrt(np.sum(jes_errs**2))
summary_errs.loc[r'$\sf jet\rightarrow\tau$',:]  = np.sqrt(np.sum(tau_misid_errs**2))

summary_errs = summary_errs.divide(params_init[:4]/100, axis=1)
summary_errs.to_latex('local_data/summary_errors.tex', escape=False)
summary_errs.to_csv('local_data/summary_errors.csv')
summary_errs